# Fashion Item Recognition
***
## Table of Contents

In [5]:
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

## 1. Introduction

## 2. Loading Data

In [ ]:
train_data = datasets.FashionMNIST(
    root="_datasets",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None,
)
test_data = datasets.FashionMNIST(
    root="_datasets", train=False, download=True, transform=ToTensor()
)

In [15]:
print(
    f"Data size \n X_train: {len(train_data.data)}, X_test: {len(test_data.data)}\n y_train: {len(train_data.targets)}, y_test: {len(test_data.targets)}"
)

Data size 
 X_train: 60000, X_test: 10000
 y_train: 60000, y_test: 10000


In [9]:
image, label = train_data[0]
label

9

In [10]:
image.shape

torch.Size([1, 28, 28])

In [16]:
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']